# Tutorial 6: Basics of netCDF file manipulation

In previous tutorials, we've worked with discrete data such as hourly air quality measurements. In this tutorial, we'll move beyond discrete data into continuous or gridded data. Specifically, we'll use satellite data, but this method can also be used with model data, reanalysis data, or any dataset that is continuous or gridded over the area of interest.   

By the end of this tutorial, you will be able to:
* open netCDF files with Xarray
* distinguish between code for DataArrays and Datasets
* identify the properties of an Xarray DataArray and Dataset
* create a plot using continuous or gridded data
* manipulate data using array properties

### How to deal with 2D or 3D data in Python

So far, we've worked with lists and tables, which are examples of 1D (lists) and simple 2D (tables) data. But what if we have more complex 2D data, or even 3D data? pandas can no longer handle this type of information. Instead, we use other packages like *Xarray*.

In [ ]:
# for data manipulation and opening files
import xarray as xr
import pandas as pd

# for plotting
import matplotlib.pyplot as plt
from cartopy import crs, feature

<font color=red> **Note:**</font> If you get an error message saying that you do not have the necessary libraries, such as h5netcdf or pseudonetcdf, then you will need to install them. To do this, follow these instructions:
1. Open either a new terminal window (Mac) or Anaconda prompt (PC).
2. Copy and paste `conda install -c conda-forge netCDF4`.
3. Hit enter.
4. When prompted, hit `y`.
5. Once the installation is complete, repeat steps 2-4 for all necessary libraries. These may include h5netcdf, PyNIO, h5py, and PseudoNetCDF. If they are already installed, conda will tell you the package is up-to-date.

### Downloading data

Files of large gridded datasets can be too large to easily share. Instead, you will need to download the data yourself. In later tutorials, we will show ways to download data with Python code and ways to work with cloud-based data without downloading. 

**Note:** Satellite data available online can be found at different levels of processing. For this tutorial, we will be working with L3, or level 3, satellite data. This means the data is spaced evenly on a uniform grid, and the time steps are uniform as well.  When searching for data, be sure to specify L3 in the search bar.

1. Go to https://worldview.earthdata.nasa.gov/. 
2. Play around with the different data available. There are many different satellite data products. For this tutorial, we will be using Aqua MODIS Global Mapped Chlorophyll-a. The code examples will be specific to this data, but you can choose any data product and just change the variable names.    
3. Once you have selected a data product and date, click "Data" in the top left corner then click on "NASA's Earthdata Search". Or go to https://search.earthdata.nasa.gov/. 
4. In the EarthData search, you will need to fill in the "Search for collections or topics" bar at the upper left corner. Include information such as the instrument (Aqua), the data product (chlorophyll), and the level (L3). Hit enter.
    - **Reminder:** Even if you choose to work with a different data product, be sure to select L3 data, not L2. 
5. In the center white panel, a list of types of satellite data products will appear. Scroll through to find the product that matches what you want. The data in this tutorial can be found under the "Aqua MODIS Global Mapped Chlorophyll (CHL) Data, version 2018" list item. Click the data product you want. 
6. A series of boxes for each data file will appear. In the case of the Aqua Chlorophyll-a product, each box is one day of data mapped globally. There are two different grid scales to choose from for each day: 4 km and 9 km grids. The 4 km grid is used in this tutorial. Once you find a date of interest and an appropriate grid size, click the download icon in the bottom lefthand corner of the data box (not the large download buttom in the bottom right corner).
7. You will be directed to register an account with NASA Earthdata. Follow the instructions to create an account. 
8. Once you've created an account and logged in, your file will automatically download.
9. Your file might download as a .zip if it is especially large. In that case, click the file to unzip and then change the file names in the tutorial to include the path.

### Opening files with xarray

In [ ]:
file = 'A2022185.L3m_DAY_CHL_chlor_a_4km.nc'

In [ ]:
# use xarray to open the file you chose
data = xr.open_dataset(file,engine='netcdf4')

Xarray can be described as a multi-dimensional version of pandas. Like pandas, the data can be indexed and sorted based on location or name. However, Xarray can do these things with data of three dimensions or more. Let's explore some of the ways Xarray organizes the data.

In [ ]:
# what does this data look like?
data

Ok, there's a lot going on here. Let's break these different things down a bit.    

**Array Type:** The very first thing that is shown in the above print out is 'xarray.Dataset'. There are two types of arrays that xarray can handle: DataArrays and Datasets. A Dataset is a collection of DataArrays. Imagine that a DataArray is a rubix cube. It is a 3D (or 2D or 4D or however shaped) array of data. A Dataset would therefore be a collection of rubix cubes. Imagine you ordered a shipment of 30 rubix cubes. Each individual rubix cube is a DataArray, and the box that all the cubes were shipped in is the Dataset. We can select one DataArray using its variable name. There are some data selection methods that only work on DataArrays, which is why it is important to distinguish between DataArrays and Datasets.

**Dimensions:** The dimensions are the directions that the file has, or the names of the axes. There are four dimensions listed in this file: lat, lon, rbg, and eightbitcolor. Since this is satellite data, we can think of these as latitudes or rows and longitudes or columns. There are 4320 rows, which means there are 4320 lines of latitude in the grid. There are 8640 columns, or 8640 lines of longitude. The rgb dimension is 3, meaning that there are 3 primary colors used to design the satellite image. Think like the 3 colors used to light up TV or computer screen pixels. Finally, there is eightbitcolor dimension. This is from the 8 bit color storage method, which is a way of storing image information in a computer. Both rgb and eightbitcolor are used in the palette Data variable to tell us how to color the satellite image.  

**Coordinates:** Coordinates are labels for each step of a dimension. This particular file has two coords: lat and lon. Each step in the lat coord has a new value (the latitude coordinate on a map), and each step in the lon coord has a new value (the longitude coordinate on a map).  

**Data Variables:** These are the variables of interest. This is the actual data in your file. For example, in this file there is "chlor-a" that might be of interest to us. In some datasets, the data variables have unusual names, which can make understanding the data difficult. xarray is great because it allows you to have metadata for each data variable, which might help to explain the data a little more. Click on the picture of a piece of paper next to the "chlor-a" variable name. What does it say? Some of the data variables have useful metadata under this piece of paper.    
Next, click on the picture of 3 stacked circles beside a data variable. What should appear is an array of the data. These are the actual values of this data variable.  

**Attributes:** Finally, Xarray has more metadata stored in the Attributes. Some of this information can be helpful, such as the time_coverage_start or the map_projection = Equidistant Cylindrical. Knowing the start and end dates of your data is super important, and knowing the map projection is important for properly formatting the data for visualization.

If you are interested in knowing just the attributes or just the coordinates or just the dims of an Xarray DataArray, you can call this information specifically.

In [ ]:
# here, we call the dimensions using the key word "dims"
data.dims

In [ ]:
# here, we call the coordinates using the key word "coords"
data.coords

In [ ]:
# here, we use the key word "attrs". Can you guess what this key word calls?
data.attrs

In [ ]:
# here, we call the data variables using the key word "var"
data.var

### Selecting and indexing data

Remember earlier when we said that DataArrays and Datasets are different? Indexing and selecting data is one of the places that distinguish DataArrays and Datasets. Right now, we have a Dataset. This is a collection of DataArrays. We can select one of these arrays using the variable name.

In [ ]:
# get a DataArray from our Dataset
# like using column names in pandas, we can select specific DataArrays using the array variable name
var = data['chlor_a']

In [ ]:
# look at the data
var

**Knowledge Check:** How is this print out different from the print out of data? Why is it different? Take a moment here to really think through the differences between var and data and what they mean. Some information you might want to think about for the next few code cells:
1. What type array is this?
2. How many dimensions are there?

The biggest difference between var and data is that var is only one variable from the dataset, but data has all the data variables. 

Like pandas, there are several ways to select data in Xarray. We can select the dimension of the data using positions or using names. Using positions, this is like saying "I want the data in axis 1". Using names, this is like saying "I want the data in the longitude". For either of these methods, you can use an integer or a label to select the index. We'll try a few examples below.

In [ ]:
# positional indexing of dimensions using integers
# translation: use a number to select the axis
var[:,0]

In the example above, we used square brackets ```[]``` to say "Take the dims of this DataArray." We then used colons ```:``` to say "Take all points in this dimension" and an integer ``0`` to say "take the 0th point in this dimension." The code ```var[:,0]``` therefore is saying "Take all values in the rows, but only in the 0th column." You can see how this was done by looking at the new dims of the data.

In [ ]:
# dimension name indexing using integers
var.isel(lon=0)

In this example, we used the `isel` (integer select) command to select only the 0th layer of the DataArray. This is the same result as the first example.

In [ ]:
# another way to index using dimension names and integers
var[dict(lon=0)]

You should now be able to see how Xarray, like pandas, has multiple methods to select and index data. This is only true for DataArrays. Datasets, because they are more complex, cannot be indexed using positions. Dimension names must be used. 

Next, let's slice our data.  
A slice is what it sounds like: it is a piece of the data that we are cutting out from the whole. `slice(None, 3)` says "Cut a piece of this data, starting at the 0th position (None) and ending at the 3rd position (3)." We used the dimension name "lon" to tell xarray which dimension to index on. So `lon=slice(None,3)` is saying "Cut a piece of the lon data, starting at the 0th position and ending at the 3rd position." This is how we cut out the first 3 lons of the data.   
In the cell below, we print out the dimensions of the original data and of some sliced data. Do the dimensions printed match your expectations?

In [ ]:
print(data.dims)
print(data[dict(lat=slice(None, 30))].dims)
print(data[dict(lon=slice(25,100))].dims)

**Knowledge Check:** If you see code that has square brackets `data[:]`, is that code selecting data using positional indexing or dimension name indexing? Which type of array can use this indexing method?

Like pandas, `[:]` indicates you are using positional indexing. The brackets say "of all the data, select based on the following positions", and the colon says "go from this first position to this last position when selecting the data". In Xarray, only a DataArray can use positional indexing. For a Dataset, you must use dimension names. 

### Data visualization

Now that we know how to select the data of interest, let's work on visualizing our data. Xarray has built-in plotting abilities, but these abilities are shortcuts of the same Matplotlib commands we learned in tutorials 3 and 4. To be more explicit in how we are visualizing and manipulating the data, we will continue to use the Matplotlib commands.

In [ ]:
# select a variable of interest for plotting
chlor = data['chlor_a']

In [ ]:
# look at the data
chlor

This data is the concentration of chlorophyll-a in the ocean as measured by reflection captured by the MODIS instrument on the Aqua satellite. The dimensions of the data are lat and lon, there is no height dimension. Therefore, we could plot this data of rows (latitudes) and columns (longitudes) using a grid that is 4320 x 8640, or we could make line plots of average concentrations over one of these dimensions. 

If you're having a hard time visualizing the data at this point, it might be helpful to go back to NASA WorldView and plot your data on the map. Go to https://worldview.earthdata.nasa.gov/ and search through the +Layers option to find your data. Then use the scroll bar at the bottom left to change the date. 

What if we wanted to know the average concentration along a latitude or longitude? Perhaps we have the hypothesis that chlorophyll-a concentration is higher at lower latitudes closer to the equator. How would we test this hypothesis? We could take the mean of the data along the axis of interest.

The chlorophyll-a data is formatted in a 2D array, and we want to take the average along one of these dimensions. If we want to know the average at each latitude, then we must take the mean along the column dimension. If this is hard to visualize, think back to a rubix cube. We want to know the average at each point along the y-axis (the rows). In order to do this, we must smush the x-axis, so take the average of the columns. 

Once again, we can using different indexing methods in Xarray to accomplish the same task. Now let's plot the averages, this time using Xarray plotting.

In [ ]:
# find the mean of the rows (latitudes) using integers
lat_means = chlor.mean(axis=1)
lat_means.dims

In [ ]:
# find the mean of the rows (latitudes) using dimension names
lat_means2 = chlor.mean('lon')
lat_means2.dims

In [ ]:
fig = plt.figure(figsize=(10,6))
lat_means.plot()
plt.show()

Conveniently, Xarray plotting can use the dimensions and the data variables to correctly label the axes. However, these labels aren't the best for professional presentations, so you can still use Matplotlib commands to clean up the figure.

In [ ]:
fig = plt.figure(figsize=(10,6))
lat_means.plot(c='k',label='July 4, 2022') # you can change the label data for your data
plt.xlabel('Latitude (˚)',fontsize=12)
plt.ylabel('Chlorophyll-a Concentration (mg/m$^{3}$)',fontsize=12)
plt.legend(fontsize=12)
plt.show()

### Merging data

So far, we've been manipulating data from one Dataset of data collected on one date. What if we want to look at data from more than one date? We downloaded the data for an entire month, can we merge two or more dates together?

In [ ]:
# first, open another dataset
file2 = 'A2022184.L3m_DAY_CHL_chlor_a_4km.nc'

In [ ]:
# use xarray to open the file you chose
data2 = xr.open_dataset(file2,engine='netcdf4')

In [ ]:
# look at the data
data2

At first glance, this Dataset looks pretty similar to the first Dataset. The dimensions, coordinates, and data variables are the same. Let's combine this Dataset with the first Dataset, but create a new "date" dimension.

In [ ]:
merge = xr.concat([data,data2],dim='date')
merge

We now have one larger Dataset that includes data and data2. These two original Datasets are differentiated by their position in the date dimension. We can label the dates by assigning coordinates.

In [ ]:
# the labels of the coords can be pandas datetimes
# change dates for your data, and comment out the methods that don't work for you

# if you have consecutive days, you can use pd.date_range
dates = pd.date_range('07-04-2022','07-05-2022') 

# if you have non-consecutive days, but if they repeat in a sequence, you can still use pd.date_range
# just specify the frequency of the day intervals
dates = pd.date_range('07-04-2022','07-05-2022',freq='1D') # change the number before D (day)

# if you have completely random days, you can make a list of dates then use pd.to_datetime
date_list = ['07-04-2022','07-05-2022']
dates = pd.to_datetime(date_list)

In [ ]:
dates

In [ ]:
merge = merge.assign_coords(date=dates)
merge.coords

There are now 5 total dimensions in the Dataset, with "date" being the first dimension. This new Dataset is the combination of the two individual Datasets. In the following exercises, you will practice indexing this new Dataset and merging another file. 

### Exercises

**1.** Select a subset of data from the merge Dataset that is   
a) all dates, the first 200 rows, and all columns

b) all dates, all rows, columns 100 to 200

c) all dates, the 3rd row, columns 20 to 50

**2.** Select the chlor_a data from the merge Dataset. Find the following means of this DataArray.   
a) Mean of the rows on each day (2 x 4320 data points)

b) Mean of the columns on each day (2 x 8640 data points)

c) Mean of each day (2 data points)

**3.** Open a third Dataset for a new date. Merge this Dataset with the first two Datasets along the date dimension. There are a few ways to do this, but the easiest way is to create a new Dataset using the `concat` command and list all three date files in the brackets.

In [ ]:
# open a new Dataset


In [ ]:
# merge the three date Datasets together


**4.** Find the daily mean chlorophyll-a concentration from the 3 merged dates. Plot a line plot for chlor_a concentration versus date.

In [ ]:
# find the mean


In [ ]:
# plot the data


### Possible Solutions

In [ ]:
# 1a
subset = merge[dict(lat=slice(None,200))]
subset.dims

In [ ]:
# 1b
subset = merge[dict(lon=slice(100,200))]
subset.dims

In [ ]:
# 1c
subset = merge[dict(lat=3,lon=slice(20,50))]
subset

In [ ]:
# 2
var = merge['chlor_a']

In [ ]:
# 2a
var.mean(axis=2)

In [ ]:
# 2b
var.mean(axis=1)

In [ ]:
# 2c
var.mean(['lat','lon'])

A quick explanation of why we are averaging along axis 2 for the rows and axis 1 for the columns: the date dimension because axis=0 when we merged the datasets together. So averaging along axis=0 would give us the average in time (4320 x 8640 grid). Lat became axis=1, and we know that latitude is the y-axis in real life. Lon became axis=2, and we know that longitude is the x-axis in real like. So to find the mean of the rows, we need to average along the longitudes, or along axis=2. To find the mean of the columns, we average along the latitudes, or along axis=1.

In [ ]:
# 3
file3 = 'A2022183.L3m_DAY_CHL_chlor_a_4km.nc'
data3 = xr.open_dataset(file3,engine='netcdf4')

In [ ]:
bigmerge = xr.concat([data,data2,data3],dim='date')
bigmerge.dims

In [ ]:
# 4
chlor = bigmerge['chlor_a']
daily = chlor.mean(['lat','lon'])

In [ ]:
fig = plt.figure(figsize=(10,8))
#daily.plot(c='k')
plt.xlabel('Date',fontsize=12)
plt.ylabel('Chlorophyll-a (mg/m$^{3}$)',fontsize=12)
plt.xticks(range(3),pd.date_range('07-02-2022','07-04-2022'))
plt.title('Daily Chlorophyll-a Concentration',fontsize=18)
plt.show()